# ADVANCED VAEs

In this section, we present further evolutions of standard variational auto-encoding approaches that are possible with the `vschaos` library. 

## Normalizing Flows

Despite the good performances of standard variational auto-encoders, reconstructions can be sub-optimal because of several factors. Among the possible issues that can occur during training, having diagonal normal distributions as variational distribution can fail to recover the real posterior, thus weakining the overall process. 

To circumvent this possible issue, Rezende & al. propose to use [Normalizing Flows](https://arxiv.org/abs/1505.05770) to increase the modelling capacity of the variational distributions. Normalizing Flows are a sequence of invertible transformations, that are able to transform simple distributions into arbitrarily complex ones (or inversely) while maintaining the likelihood of the output sample tractable.
In `vschaos`, this can be easily done using the `vschaos.modules.flow` library, implementing a set of different normalizing flows. This is parameterized directly in the latent parameters of the VAE, using the `Flow` wrapper that wraps a `NormalizingFlow` and a distribution, allowing direct sampling and log likelihood.

In [ ]:
import torch, pdb
import vschaos.vaes as vaes
import vschaos.distributions as dist
import vschaos.modules.flow as flow
from vschaos.data.data_generic import dataset_from_torchvision
from vschaos.data.data_transforms import Squeeze, Flatten
from torchvision.transforms import Lambda, ToTensor

transforms = [Lambda(lambda x: x / 255. + 0.01*torch.randn_like(x.float())), Flatten(-2)]
dataset = dataset_from_torchvision('MNIST', transforms=transforms)

input_params = {'dim':784, 'dist': dist.Normal}
hidden_params = {'dim':800, 'nlayers':2}
# Normalizing flows are naturally handled by the vschaos modules, and can be used by linking a distribution
#   with a flow in the latent signature
flow = flow.NormalizingFlow(8, [flow.PlanarFlow], 4)
latent_params = {'dim':8, "dist":dist.Flow(dist.Normal, flow)}

vae = vaes.VanillaVAE(input_params, latent_params, hidden_params=hidden_params)
x, y = dataset[:10]
out = vae.encode(x)

print(out[0]['out_params'])
out, out_preflow = out[0]['out_params'].sample()
print(out)

## Multi-layered VAEs

Latent layers can be stacked to obtain multi-layered VAEs (see [Rezende & al.](https://arxiv.org/pdf/1401.4082)). This is naturally handled by `vschaos` specifying latent parameters as lists, each dictionary defining a different latent space in increasing order (for the `VanillaVAE` class ; can be specific to model). Even if hierarchical layers can harden the training of the auto-encoder, it can sometimes be useful for semi-conditioning (semi-supervised models, [see Kingma & al.](http://papers.nips.cc/paper/5352-semi-supervised-learning-withdeep-generative-models)) or specific disentangling. 

In [ ]:
import torch, pdb
import vschaos.vaes as vaes
import vschaos.distributions as dist
from vschaos.data.data_generic import dataset_from_torchvision
from vschaos.data.data_transforms import Flatten
from torchvision.transforms import Lambda, ToTensor
import vschaos.criterions as crit


transforms = [Lambda(lambda x: x / 255. + 0.01*torch.randn_like(x.float())), Flatten(-2)]
dataset = dataset_from_torchvision('MNIST', transforms=transforms)

input_params = {'dim':784, 'dist': dist.Normal}
hidden_params = [{'dim':800, 'nlayers':2}, {'dim':400, 'nlayers':1, 'linked':True}, {'dim':200, 'nlayers':1, 'linked':True}]
latent_params = [{'dim':16, 'dist':dist.Normal},
                 [{'dim':4, 'dist':dist.Normal}, {'dim':4, 'dist':dist.Normal}],
                 [{'dim':2, 'dist':dist.Normal}, {'dim':2, 'dist':dist.Normal}]]

vae = vaes.VanillaVAE(input_params, latent_params, hidden_params=hidden_params)
print(vae)

# different losses can be set by giving ELBO modules a list as regularization losses
reconstruction_loss = crit.MSE()
regularization_loss = [crit.KLD(), 0.5*crit.KLD(), 0.2*crit.KLD()]
loss = crit.ELBO(reconstruction_loss=reconstruction_loss, regularization_loss=regularization_loss)

x, y = dataset[:10]
out = vae.forward(x, y=y)
current_loss, current_losses = loss.loss(model=vae, out=out, target=x, y=y)

print(out['z_params_enc'][0], out['z_params_enc'][1], out['z_params_enc'][2])
print(loss.get_named_losses(current_losses))


## Alternative divergences

Since the original variational auto-encoders, several alternative models have been then proposed to alleviate some weaknesses, notably by proposing alternative divergences for latent space regularization. Some of them are available in `vschaos`, allowing to model (and even mix) several of these alternative models. Notably : 

* the *Wasserstein auto-encoder* (WAE), proposed by [Tolstikhin & al.](https://arxiv.org/pdf/1711.01558.pdf)
* Rényi divergence variational inference, proposed by [Li & al.](https://papers.nips.cc/paper/6208-renyi-divergence-variational-inference.pdf)
* the *Adversarial auto-encoder* (AAE), proposed by [Makhzani & al.](https://arxiv.org/pdf/1511.05644.pdf%5D) (next version)

and others to come. Renyi divergences and Jensen-Shannon divergences are formulated using the `kl_divergence`formalism of the `torch` package, but are so far only available for two diagonal distributions. Do not hesitate to contribute!


In [ ]:
input_params = {'dim':784, 'dist': dist.Normal}
hidden_params = [{'dim':800, 'nlayers':2}, {'dim':400, 'nlayers':1}, {'dim':200, 'nlayers':1}]
latent_params = [{'dim':16, 'dist':dist.Normal}, {'dim':6, 'dist':dist.Normal}]

# The Wasserstein auto-encoder can be created by using the MMD criterion
regularization_loss = crit.MMD
wae = vaes.VanillaVAE(input_params, latent_params, hidden_params=hidden_params)
loss = crit.ELBO(reconstruction_loss=reconstruction_loss, regularization_loss=regularization_loss)
out = wae.forward(dataset[:10][0])
print(loss.get_named_losses(loss(model=wae, out=out, target=dataset[:10][0])[1]))

# The Rényi auto-encoder can be created by using the RD criterion
regularization_loss = crit.RD(alpha=2.0)
rae = vaes.VanillaVAE(input_params, latent_params, hidden_params=hidden_params)
loss = crit.ELBO(reconstruction_loss=reconstruction_loss, regularization_loss=regularization_loss)
out = rae.forward(dataset[:10][0])
print(loss.get_named_losses(loss(model=rae, out=out, target=dataset[:10][0])[1]))
